In [15]:
import pandas as pd
import datetime

In [3]:
data = pd.read_csv('IDPHExport.csv')

In [19]:
data.head()

,CountyName,AdministeredCount,AdministeredCountChange,AdministeredCountRollAvg,PersonsFullyVaccinated,Report_Date,PctVaccinatedPopulation
0,Illinois,1,1,0,0,12/15/20 0:00,0.0
1,Illinois,1995,1994,285,0,12/16/20 0:00,0.0
2,Illinois,13035,11040,1862,0,12/17/20 0:00,0.0
3,Illinois,34047,21012,4864,0,12/18/20 0:00,0.0
4,Illinois,47297,13250,6757,0,12/19/20 0:00,0.0


In [9]:
data.dtypes

CountyName                   object
AdministeredCount             int64
AdministeredCountChange       int64
AdministeredCountRollAvg      int64
PersonsFullyVaccinated        int64
Report_Date                  object
PctVaccinatedPopulation     float64
dtype: object

In [ ]:
data["datetime"] = ""

for index, row in data.iterrows():
    
    date = row["Report_Date"]
    
    formatted = datetime.datetime.strptime(date, '%-m/%-d/%y %-I:%M')
    
    dna_data.loc[index, "Sample Month"] = formatted

In [17]:
pd.to_datetime(data["Report_Date"], format='%-m/%-d/%y %-I:%M')



ValueError: '-' is a bad directive in format '%-m/%-d/%y %-I:%M'